# Интеграция результатов в единый пайплайн

<b>Цель:</b> Собрать единый пайплайн от загрузки и очистки данных до классификации и записи эмбеддингов в Qdrant.

<b>Реализация: </b> В этом ноутбуке я буду экспериментировать с настройкой пайплайна для 2х разных сеттингов: офисный, в котором можно работать на GPU c большими данными и мой локальный, в котором можно быстро тестировать некоторые гипотезы и методы. По итогам этой работы я также подготовлю скрипт для работы на больших данных на кластере.

Пайплайн: 
1. На офисном компьютере:
    - Чтение данных из MongoDB 
    - Предвариательная обработка (очистка от двойных пробелов, нечитаемых символов, стоп-слов; от лемматизации отказались) 
    - Векторизация 
    - Классификация (большими моделями на GPU)
    - Сохранение результатов в векторную базу Qdrant.
2. На моём ноутбуке:
    - Чтение небольших сэмплов данных из csv 
    - Предвариательная обработка (очистка от двойных пробелов, нечитаемых символов, стоп-слов; от лемматизации отказались) 
    - Векторизация 
    - Классификация ("лёгкими" моделями, т.к. у меня нет GPU)
    - Сохранение результатов в csv

## Сруктура ноутбука

1. Настройка среды
2. Импорты библиотек и инициализация
3. Загрузка данных
4. Предобработка
5. Создание эмбеддингов
6. Сохранение результатов

## 1. Настройка среды

В этой части кода мы даём ноутбуку понять, где мы работаем, через флаг ENV:
- `ENV = "local"` - работаем на моём локальном ноутбуке. Из config.yaml в корне будут считаны следующие параметры:
    - `DATA_PATH: "./data/sample.csv"` - локальный путь к csv файлу с сэмплом данных на диске
    - `USE_GPU: false` - не использовать GPU, потому что на этом компьютере её нет
    - `EMBEDDING_MODEL: "all-MiniLM-L6-v2"` - использовать "лёгкую" модель для эмбеддинга, которую комп потянет и без GPU
- `ENV = "prod"` - работаем в офисном сеттинге. Из config.yaml в корне будут считаны следующие параметры:
    - `DATA_PATH: null` - данные будут считываться из mongoDB и писаться в Qdrant. `MONGO_URI` и `QDRANT_URL` указаны в файле `.env` в корне репозитория. 
    - `USE_GPU: true` - в проде есть GPU, и мы будем её использовать  
    - `EMBEDDING_MODEL: "sentence-transformers/all-mpnet-base-v2"` - используем более "продвинутую" LLM для эмбеддингов.

In [3]:
import sys
import os

# Абсолютный путь к корню проекта
project_root = os.path.abspath(os.path.join(".."))  # если notebooks/ внутри корня
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils.config_loader import load_config

ENV = "local"           #ПЕРЕКЛЮЧИТЬ НАСТРОЙКУ ЗДЕСЬ!
# ENV = "prod"
CONFIG = load_config(ENV, project_root=project_root)

print(f"Running in {ENV.upper()} mode")
print(CONFIG)


Running in LOCAL mode
{'DATA_PATH': './data/sample.csv', 'USE_GPU': False, 'EMBEDDING_MODEL': 'all-MiniLM-L6-v2', 'MONGO_URI': 'mongodb://localhost:27017', 'QDRANT_URL': 'http://localhost:6333'}


## 2. Импорты библиотек и инициализация

In [4]:
import pandas as pd
from pymongo import MongoClient
from tqdm.auto import tqdm
import re
import spacy
from transformers import pipeline
import matplotlib.pyplot as plt